In [34]:
import csv
import random
import time
import json

In [35]:
with open("/home/zjusst/qms/webarena/result_stage_1_explore_v2/flitered_triplets.csv", "r") as f:
    reader = csv.reader(f)
    # 跳过第一行
    next(reader)
    triplets = [row for row in reader]
len(triplets)

1028

In [36]:
out_degree_dict = {}
for triplet in triplets:
    if triplet[3] not in out_degree_dict:
        out_degree_dict[triplet[3]] = 0
    out_degree_dict[triplet[3]] += 1

# 转为 list，由低到高排序
out_degree_list = sorted(out_degree_dict.items(), key=lambda x: x[1])
out_degree_list[0]


("('812_1.json', 2, 1, 'http://reddit.com/create_forum')", 1)

In [37]:
flitered_out_degree_list = []
for item in out_degree_list:
    real_url = eval(item[0])[3]
    if real_url.startswith("http://reddit.com/"):
        flitered_out_degree_list.append(item)

len(flitered_out_degree_list)

563

In [38]:
flitered_out_degree_list[0]

("('812_1.json', 2, 1, 'http://reddit.com/create_forum')", 1)

In [39]:
prefix_to_record_dict = {}
for record in flitered_out_degree_list:
    real_url = eval(record[0])[3]
    # 去掉 http://reddit.com/ ，拿到后面部分
    prefix = real_url.split("reddit.com/")[1]
    if prefix == "": 
        continue
    # 如果 prefix 还存在 / 或者 ? ，取前面的部分
    if "/" in prefix:
        prefix = prefix.split("/")[0]
    if "?" in prefix:
        prefix = prefix.split("?")[0]
    if prefix == "wiki":
        continue
    if prefix not in prefix_to_record_dict:
        prefix_to_record_dict[prefix] = []
    prefix_to_record_dict[prefix].append(record)
prefix_to_record_dict.keys()

dict_keys(['create_forum', 'submit', 'search', 'f', 'forums', 'user', 'comments', 'new', 'notifications', 'messages', 'tags'])

In [40]:
def random_select_url_and_a11y(prefix_to_record_dict):
    # 种子为时间
    random.seed(time.time())
    # 随机选择一个 prefix
    prefix = random.choice(list(prefix_to_record_dict.keys()))
    record = prefix_to_record_dict[prefix][random.randint(0, len(prefix_to_record_dict[prefix]) - 1)]
    file_name, traj_index, pos, real_url = eval(record[0])
    with open(f"/home/zjusst/qms/webarena/result_stage_1_explore_v2/add_local_intention_trajs/{file_name}", "r") as f:
        trajs = json.load(f)
    traj = trajs[traj_index]
    if pos == 0:
        a11y = traj["a11y_before"]
    else:
        a11y = traj["a11y_after"]
    return real_url, a11y

In [41]:
random_select_url_and_a11y(prefix_to_record_dict)[0]

'http://reddit.com/search?q=forum+exploration'

In [42]:
def random_select_webarena_intension(k):
    intensions = ['Like all submissions created by {{user}} in subreddit {{subreddit}}',
        #'Ask for advice about {{issue}} in a subreddit for relations',
        'Among the top {{number}} post in "{{subreddit}}" forum, {{description}}',
        #'Post in the most appropriate subreddit and ask for recommendations for {{category}} products within a budget of {{price}}',
        'Change my reddit bio to "{{content}}"',
        #'Re-post the image of {{content}} in this page to {{subreddit}} subreddit and note "from /f/pics"',
        #'Post a review of my recent reading "{{book}}" in the r/books with my comment "{{content}}".',
        #'Post in {{subreddit}} forum about what could large language models help the correpong field.',
        'Upvote the newest post in {{subreddit}} subreddit',
        'DisLike all submissions created by {{user}} in subreddit {{subreddit}}',
        #'Post a notice on a virtual meetup for {{interest}} enthusiasts on {{date}} in the {{subreddit}} subreddit',
        'Open the thread of a trending post on the forum "{{subreddit}}" and subscribe.',
        #'Post in {{subreddit}} subreddit about what could machine learning help the correpong field.',
        #'Create a discussion post about "{{topic}}" in a relevant subreddit and ask users for their opinions with the simple prompt, "your opinion"',
        #'Tell me the count of comments that have received more downvotes than upvotes for the user who made the latest post on the {{forum}} forum.',
        #'Ask for product recommendations for {{category}} within a budget of {{price}} in {{subreddit}}',
        #'Reply to {{position_description}} with my comment "{{content_description}}"',
        'Create a new forum named {{name}}, with a description of {{description}}, and include {{sidebar_list}} in the sidebar?',
        #'Post my question, "{{question}}", in a subreddit where I\'m likely to get an answer',
        #'Find a subreddit focused on topics related to {{topic}}, and post my question, "{{question}}" there',
        'Edit my post on {{post}} by adding a line to the body that says "{{content}}"',
        'Thumbs down the top {{k}} post ever in {{subreddit}}.',
        #'Reply to {{position_description}} in this post with "{{content_description}}"',
        #'Post in {{subreddit}} forum about what could open-source LLMs help the correpong field.',
        #'Post in {{subreddit}} subreddit about what could diffusion model help the correpong field.',
        #'Post in {{subreddit}} subreddit about what could midjourney help the correpong field.'
    ]
    random.seed(time.time())
    # 随机选择 k 个
    return random.sample(intensions, k)
    

In [43]:
random_select_webarena_intension(5)

['Among the top {{number}} post in "{{subreddit}}" forum, {{description}}',
 'Upvote the newest post in {{subreddit}} subreddit',
 'Edit my post on {{post}} by adding a line to the body that says "{{content}}"',
 'DisLike all submissions created by {{user}} in subreddit {{subreddit}}',
 'Change my reddit bio to "{{content}}"']

In [44]:
def random_persona(personas):
    random.seed(time.time())
    return personas[random.randint(0, len(personas) - 1)]

In [45]:
personas = []
# with open("/home/zjusst/qms/persona-hub/data/elite_personas_10000.jsonl", "r") as f:
with open("/home/zjusst/qms/persona-hub/data/persona.jsonl", "r") as f:
    for line in f:
        persona = json.loads(line)
        personas.append(persona['persona'])

In [46]:
persona = random_persona(personas)
persona

'A marketing manager who wants to create a memorable brand activation at a trade show'

In [47]:
real_url, a11y = random_select_url_and_a11y(prefix_to_record_dict)
real_url, a11y[:100]

('http://reddit.com/f/allentown/2/sample-title',
 "Tab 0 (current): Sample Title\n\n[2746] RootWebArea 'Sample Title' focused: True\n\t[2788] HeaderAsNonLa")

In [48]:
system_prompt = '''You are an expert AI specializing in simulating realistic human-computer interaction. Your mission is to generate a single, high-quality user goal based on a given Persona and a Seed Page from a website. This goal will be used to guide an autonomous agent in exploring a website.

Note: 
1. Your response MUST begin with the exact phrase `As a [Persona], my goal is to...`. The goal you generate should be a natural and logical objective for someone with that persona's background, needs, and motivations.
2. Create a Challenging and Informative Goal. The goal should be non-trivial and likely require a sequence of logical steps (e.g., navigating, searching, comparing, finding specific information). Avoid simple, single-click tasks. The goal must be concrete and specific.
3. Be Creative. Use the Persona and the Seed Page as inspiration to create a believable scenario. The seed page can be part of the goal.
4. Go beyond just post or comment actions. You can infer these example short goal to **avoid** just post or comment actions: {example_short_goal}

Your output should be a single, coherent sentence that clearly states this challenging, persona-driven goal. You must refer to the example short goal to add more specific goals to avoid just post and comment actions. Because these specific functions are also important.
'''

user_prompt_template = '''
URL: {url}
A11y: {a11y}
Persona: {persona}
'''

few_shot_examples = [
    (
        """URL: http://reddit.com/f/relationship_advice
A11y: Tab 0 (current): relationship_advice

[3036] RootWebArea 'relationship_advice' focused: True
[3049] main ''
		[3239] heading '/f/relationship_advice'
		[3053] button 'Sort by: Hot' focused: True hasPopup: menu expanded: True
		[4806] link '• Hot'
		[4803] link 'New'
		[4800] link 'Active'
		[4816] link 'Top'
		[4809] link 'Controversial'
		[4815] link 'Most commented'
		[3054] article ''
			[3057] HeaderAsNonLandmark ''
				[3298] heading 'Need advice on handling infidelity'
					[3702] link 'Need advice on handling infidelity'
				[3704] StaticText 'Submitted by '
				[3060] link 'MarvelsGrantMan136' expanded: False
				[3706] time 'October 9, 2025 at 3:48:16 AM EDT'
					[4179] StaticText '11 days ago'
			[4180] link '2 comments'
			[4182] link 'Edit'
			[4736] button 'Delete'
			[3302] button 'Retract upvote'
			[3305] button 'Downvote'

Persona: A PhD candidate in remote sensing and machine learning, specifically focused on change detection of Arctic glaciers using satellite imagery and deep learning algorithms. The candidate has experience in developing and implementing data-driven methods for landscape change analysis and has completed a Master's degree in Geography and Environmental Sciences. The candidate is interested in using cutting-edge technology and machine learning techniques to understand and predict changes in Arctic glaciers, with the ultimate goal of informing and engaging communities about the impacts of climate change. The candidate is a strong researcher with a focus on developing innovative tools and user-friendly mapping solutions for the Arctic region.""",
    #'As a PhD candidate in remote sensing and machine learning, my goal is to find a subreddit focused on topics related to geospatial data science or climate change, and post my question, "What open-source LLMs are best suited for analyzing and summarizing satellite imagery time-series data for Arctic glacier change detection?"'
	'As a PhD candidate in remote sensing and machine learning, my goal is to change by bio, browse posts about remote sensing and machine learning, follow a forum matches my interest, and give likes to good posts.'
    ),
    (
        """
URL: http://reddit.com/f/worldnews
A11y: Tab 0 (current): worldnews

[2827] RootWebArea 'worldnews' focused: True
[2845] main ''
		[3030] heading '/f/worldnews'
		[3540] link 'Submissions'
		[3541] link 'Comments'
		[2849] button 'Sort by: Hot' hasPopup: menu expanded: False
		[2850] article ''
			[2853] HeaderAsNonLandmark ''
				[3089] heading 'Ireland Aims To Legalize Cannabis For Personal Use'
					[3543] link 'Ireland Aims To Legalize Cannabis For Personal Use'
				[3091] link 'forbes.com'
				[3546] StaticText 'Submitted by '
				[2856] link 'seebz69' expanded: False
				[4046] StaticText 't3_z6vut3'
				[3550] time 'November 28, 2022 at 7:55:11 AM EST'
					[4050] StaticText '3 years ago'
			[4051] link '201 comments'
			[3095] button 'Upvote'
			[3553] StaticText '10614'
			[3098] button 'Downvote'

Persona: A geophysicist who specializes in studying the properties and behavior of the Earth's core, particularly in relation to its composition and structure. He or she is interested in understanding how the core is formed, how it changes over time, and how it interacts with the rest of the planet. This person may have expertise in using advanced technology, such as the University of Chicago's laser-heated diamond anvil cell, to study the inner core. They may also be interested in exploring the potential for the inner core to be a mixture of different crystal phases, such as hexagonal close-packed and body-centered cubic iron, and how this could affect the Earth's magnetic field and other geological processes.""",
	"As a geophysicist who specializes in studying the properties and behavior of the Earth's core, my goal is edit my last post and add content about the inner core. And create a new forum about Earth's core if possible."
    )
]

In [49]:
# print(user_prompt_template.format(url=real_url, a11y=tmp_a11y, persona=persona))

In [50]:
# from langchain_openai import ChatOpenAI
# import os

# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# os.environ['LANGSMITH_TRACING'] = 'true'
# os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGSMITH_API_KEY'] = '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'
# os.environ['LANGSMITH_PROJECT'] = 'pr-wooden-hedgehog-65'
# os.environ['OPENAI_API_KEY'] = 'sk-khwk6tpwWCA7ANFoljLONZ4Xln766pTzuEkDkEXYvxdzNcXQ'
# os.environ['OPENAI_API_BASE'] = 'https://api2.aigcbest.top/v1'

# llm = ChatOpenAI(
#     model="gpt-4o",
#     api_key="sk-khwk6tpwWCA7ANFoljLONZ4Xln766pTzuEkDkEXYvxdzNcXQ",
#     base_url="https://api2.aigcbest.top/v1"
# )
# print(llm.invoke("hello")) 

In [51]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT'] = 'pr-wooden-hedgehog-65'
os.environ['OPENAI_API_KEY'] = 'sk-khwk6tpwWCA7ANFoljLONZ4Xln766pTzuEkDkEXYvxdzNcXQ'
os.environ['OPENAI_API_BASE'] = 'https://api2.aigcbest.top/v1'

from typing import Any, List, Dict

# LangChain 和 LangSmith 的相关导入
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ChatMessage

def generate_from_openai_chat_completion_new(
    messages: List[Dict[str, str]],
    model: str,
    temperature: float,
    max_tokens: int,
    top_p: float,
    context_length: int, # 这个参数在ChatOpenAI中不直接使用，但为了保持签名一致而保留
    stop_token: str | None = None,
) -> str:
    """
    使用 LangChain 的 ChatOpenAI 生成聊天回复，并支持 LangSmith 追踪。
    函数签名与旧版完全兼容。
    """
    if "OPENAI_API_KEY" not in os.environ and "api_key" not in locals():
        raise ValueError("OPENAI_API_KEY environment variable must be set.")

    # 1. 初始化 LangChain 的 ChatOpenAI 客户端
    # 将旧函数的参数映射到 ChatOpenAI 的构造函数中
    llm = ChatOpenAI(
        # 从环境变量获取 key 和 base_url，提供默认值
        api_key=os.environ.get("OPENAI_API_KEY"),
        base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1"),
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stop=[stop_token] if stop_token else None,
        model_kwargs={}, # 如果没有其他高级参数，可以为空字典或直接省略
        # 内置重试机制，替代旧的装饰器
        max_retries=3,
    )

    # 2. 将输入的字典列表转换为 LangChain 的消息对象
    # 这是一个必要的适配步骤
    langchain_messages = []
    for msg in messages:
        role = msg.get("role")
        content = msg.get("content", "")
        # 从字典中获取 name，如果不存在则为 None
        name = msg.get("name")

        if role == "user":
            langchain_messages.append(HumanMessage(content=content, name=name))
        elif role == "assistant":
            langchain_messages.append(AIMessage(content=content, name=name))
        elif role == "system":
            # 这会正确处理您提供的 few-shot 示例
            langchain_messages.append(SystemMessage(content=content, name=name))
        else:
            # 为其他可能的角色（如 'tool'）提供一个通用处理
            langchain_messages.append(ChatMessage(role=role, content=content))

    # 3. 调用 LLM 并获取结果
    response = llm.invoke(langchain_messages)

    # 4. 提取并返回内容，保持与旧函数相同的字符串输出
    return response.content

In [52]:
# example_messages_with_name = [
#     {
#         'role': 'user',
#         "content": "hello"
#     }
# ]
# result = generate_from_openai_chat_completion_new(
#     messages=example_messages_with_name,
#     model="gpt-4o",
#     temperature=0.7,
#     max_tokens=2000,
#     top_p=1.0,
#     context_length=4096,
#     stop_token=None
# )
# print("LLM Response:", result)

In [54]:
for i in range(30):

# 使用您提供的包含 'name' 字段的 messages 列表

    real_url, a11y = random_select_url_and_a11y(prefix_to_record_dict)
    persona = random_persona(personas)
    short_goal = random_select_webarena_intension(5)
    example_messages_with_name = [
        {
            'role': 'system',
            'content': system_prompt.format(example_short_goal=short_goal)
        },
        {
            'role': 'system',
            'name': 'example_user',
            'content': few_shot_examples[0][0]
        },
        {
            'role': 'system',
            'name': 'example_assistant',
            'content': few_shot_examples[0][1]
        },
        {
            'role': 'system',
            'name': 'example_user',
            'content': few_shot_examples[1][0]
        },
        {
            'role': 'system',
            'name': 'example_assistant',
            'content': few_shot_examples[1][1]
        },
        {
            'role': 'user',
            'name': 'user',
            'content': user_prompt_template.format(url=real_url, a11y=a11y, persona=persona)
        }
    ]

    # 调用修正后的新函数
    try:
        result = generate_from_openai_chat_completion_new(
            messages=example_messages_with_name,
            model="gpt-4o",
            temperature=0.7,
            max_tokens=2000,
            top_p=1.0,
            context_length=4096,
            stop_token=None
        )
        print("LLM Response:", result)
        with open(f"/home/zjusst/qms/webarena/result_stage_2_intended_explore/generated_intensions_v2.txt", "a") as f:
            f.write(result + "\n")
    except Exception as e:
        print(f"An error occurred: {e}")

LLM Response: As a rookie iOS developer eager to implement UI details correctly, my goal is to search for discussions on the newest iOS features, find and bookmark posts with expert advice on using APIs, and engage with the community by commenting on threads that address my specific UI challenges.
LLM Response: As a public relations manager for a sports agency representing high-profile baseball players, my goal is to explore how the features of the neograd deep learning framework can be utilized to develop advanced sports performance analytics for our athletes, subscribe to the MachineLearning forum for updates on similar innovations, and analyze posts for potential collaborations with tech developers to enhance athlete training programs.
LLM Response: As a wedding photographer who hates the cold, my goal is to explore recent comments related to wedding photography in warmer climates, find a valuable piece of advice or information from other users, and upvote it while also bookmarking 

In [60]:
import json


start_index = 813
config_json_save_dir_name = "/home/zjusst/qms/webarena/config_files"

config_template = {
  "sites": [
    "reddit"
  ],
  "task_id": 812,
  "require_login": True,
  "storage_state": "./.auth/reddit_state.json",
  "start_url": "http://10.130.138.30:9999",
  "geolocation": None,
  "intent_template": "N/A",
  "instantiation_dict": {
    "forum": "N/A"
  },
  "intent": "Default",
  "require_reset": False,
  "eval": {
    "eval_types": [
      "string_match"
    ],
    "reference_answers": {
      "must_include": [
        "0"
      ]
    },
    "reference_url": "",
    "program_html": [],
    "string_note": "",
    "reference_answer_raw_annotation": "0"
  },
  "intent_template_id": "N/A"
}

generate_instructions = []
with open(f"/home/zjusst/qms/webarena/result_stage_2_intended_explore/generated_intensions_v2.txt", "r") as f:
    for line in f:
        generate_instructions.append(line.strip())
 
for i in range(len(generate_instructions)):
    config_json = config_template
    config_json["task_id"] = start_index + i
    config_json["intent"] = generate_instructions[i]
    with open(f"{config_json_save_dir_name}/{start_index + i}.json", "w") as f:
        json.dump(config_json, f)
